# 🎓 Proyecto Capstone: Análisis de Datos Empresariales

¡Bienvenido al proyecto final! En este capstone integrarás todos los conocimientos adquiridos para realizar un análisis completo de datos empresariales.

## 📊 Descripción del Proyecto

Como analista de datos de **TechData Solutions**, tu tarea es analizar el rendimiento de ventas del primer mes de 2024 y generar insights valiosos para la dirección.

## 🎯 Objetivos del Análisis
- Analizar tendencias de ventas por producto y región
- Identificar clientes más rentables
- Evaluar rendimiento del equipo de ventas
- Generar recomendaciones estratégicas

## 1. 📥 Importación y Configuración Inicial

In [ ]:
# Importar librerías necesarias
import json
from datetime import datetime
import os

# Configuración del proyecto
EMPRESA = "TechData Solutions"
PERIODO_ANALISIS = "Enero 2024"
FECHA_REPORTE = datetime.now().strftime("%Y-%m-%d %H:%M")

print(f"📊 {EMPRESA} - Análisis de Datos")
print(f"Período: {PERIODO_ANALISIS}")
print(f"Reporte generado: {FECHA_REPORTE}")
print("=" * 50)

## 2. 📂 Carga de Datos

In [ ]:
# TODO: Implementa las funciones de carga de datos

def cargar_ventas(archivo):
    """Carga datos de ventas desde archivo CSV"""
    ventas = []
    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            lineas = file.readlines()
            headers = lineas[0].strip().split(',')
            
            for linea in lineas[1:]:
                valores = linea.strip().split(',')
                venta = {}
                for i, header in enumerate(headers):
                    venta[header] = valores[i]
                
                # Convertir tipos apropiados
                venta['cantidad'] = int(venta['cantidad'])
                venta['precio_unitario'] = float(venta['precio_unitario'])
                venta['total'] = venta['cantidad'] * venta['precio_unitario']
                
                ventas.append(venta)
                
        print(f"✅ Cargadas {len(ventas)} transacciones de ventas")
        return ventas
        
    except FileNotFoundError:
        print(f"❌ Error: No se pudo encontrar el archivo {archivo}")
        return []
    except Exception as e:
        print(f"❌ Error cargando ventas: {e}")
        return []

def cargar_clientes(archivo):
    """Carga datos de clientes desde archivo JSON"""
    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            data = json.load(file)
            clientes = data['clientes']
            
        print(f"✅ Cargados {len(clientes)} clientes")
        return clientes
        
    except FileNotFoundError:
        print(f"❌ Error: No se pudo encontrar el archivo {archivo}")
        return []
    except Exception as e:
        print(f"❌ Error cargando clientes: {e}")
        return []

def cargar_productos(archivo):
    """Carga catálogo de productos desde archivo de texto"""
    productos = {}
    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            contenido = file.read()
            secciones = contenido.split('---')
            
            for seccion in secciones:
                if 'ID:' in seccion:
                    lineas = seccion.strip().split('\n')
                    producto = {}
                    
                    for linea in lineas:
                        if ':' in linea:
                            clave, valor = linea.split(':', 1)
                            clave = clave.strip().lower().replace(' ', '_')
                            valor = valor.strip()
                            
                            if clave == 'precio':
                                # Extraer número del precio
                                valor = float(valor.replace('$', '').replace(',', ''))
                            elif clave == 'stock':
                                valor = int(valor.split()[0])
                                
                            producto[clave] = valor
                    
                    if 'id' in producto:
                        productos[producto['id']] = producto
        
        print(f"✅ Cargados {len(productos)} productos")
        return productos
        
    except FileNotFoundError:
        print(f"❌ Error: No se pudo encontrar el archivo {archivo}")
        return {}
    except Exception as e:
        print(f"❌ Error cargando productos: {e}")
        return {}

# Cargar todos los datos
ventas = cargar_ventas('datos/ventas.csv')
clientes = cargar_clientes('datos/clientes.json')
productos = cargar_productos('datos/productos.txt')

print(f"\n📊 Resumen de datos cargados:")
print(f"- Transacciones de ventas: {len(ventas)}")
print(f"- Clientes: {len(clientes)}")
print(f"- Productos: {len(productos)}")

## 3. 🔍 Análisis Exploratorio de Datos

In [ ]:
# TODO: Implementa análisis exploratorio

def analizar_ventas_generales(ventas):
    """Análisis general de ventas"""
    if not ventas:
        print("No hay datos de ventas para analizar")
        return
    
    # Métricas generales
    total_transacciones = len(ventas)
    ingresos_totales = sum(venta['total'] for venta in ventas)
    ticket_promedio = ingresos_totales / total_transacciones
    
    print("💰 ANÁLISIS GENERAL DE VENTAS")
    print("=" * 35)
    print(f"Total de transacciones: {total_transacciones}")
    print(f"Ingresos totales: ${ingresos_totales:,.2f}")
    print(f"Ticket promedio: ${ticket_promedio:,.2f}")
    
    # Análisis por producto
    ventas_por_producto = {}
    for venta in ventas:
        producto_id = venta['producto_id']
        if producto_id not in ventas_por_producto:
            ventas_por_producto[producto_id] = {
                'cantidad': 0,
                'ingresos': 0,
                'transacciones': 0
            }
        
        ventas_por_producto[producto_id]['cantidad'] += venta['cantidad']
        ventas_por_producto[producto_id]['ingresos'] += venta['total']
        ventas_por_producto[producto_id]['transacciones'] += 1
    
    print("\n📦 VENTAS POR PRODUCTO:")
    print("-" * 25)
    for producto_id, datos in ventas_por_producto.items():
        nombre = productos.get(producto_id, {}).get('nombre', 'Desconocido')
        print(f"{producto_id} ({nombre}):")
        print(f"  Unidades vendidas: {datos['cantidad']}")
        print(f"  Ingresos: ${datos['ingresos']:,.2f}")
        print(f"  Transacciones: {datos['transacciones']}")
        print()
    
    return ventas_por_producto

def analizar_por_region(ventas):
    """Análisis de ventas por región"""
    ventas_por_region = {}
    
    for venta in ventas:
        region = venta['region']
        if region not in ventas_por_region:
            ventas_por_region[region] = {
                'ingresos': 0,
                'transacciones': 0,
                'clientes_unicos': set()
            }
        
        ventas_por_region[region]['ingresos'] += venta['total']
        ventas_por_region[region]['transacciones'] += 1
        ventas_por_region[region]['clientes_unicos'].add(venta['cliente_id'])
    
    print("🗺️ ANÁLISIS POR REGIÓN:")
    print("=" * 25)
    
    for region, datos in ventas_por_region.items():
        clientes_unicos = len(datos['clientes_unicos'])
        ticket_promedio = datos['ingresos'] / datos['transacciones']
        
        print(f"Región {region}:")
        print(f"  Ingresos: ${datos['ingresos']:,.2f}")
        print(f"  Transacciones: {datos['transacciones']}")
        print(f"  Clientes únicos: {clientes_unicos}")
        print(f"  Ticket promedio: ${ticket_promedio:,.2f}")
        print()
    
    return ventas_por_region

# Ejecutar análisis
ventas_por_producto = analizar_ventas_generales(ventas)
ventas_por_region = analizar_por_region(ventas)

## 4. 👥 Análisis de Clientes

In [ ]:
# TODO: Análisis detallado de clientes

def analizar_clientes(ventas, clientes):
    """Análisis detallado de comportamiento de clientes"""
    
    # Crear diccionario de clientes por ID
    clientes_dict = {cliente['id']: cliente for cliente in clientes}
    
    # Análisis de ventas por cliente
    ventas_por_cliente = {}
    
    for venta in ventas:
        cliente_id = venta['cliente_id']
        if cliente_id not in ventas_por_cliente:
            ventas_por_cliente[cliente_id] = {
                'ingresos': 0,
                'transacciones': 0,
                'productos_diferentes': set(),
                'ultima_compra': venta['fecha']
            }
        
        ventas_por_cliente[cliente_id]['ingresos'] += venta['total']
        ventas_por_cliente[cliente_id]['transacciones'] += 1
        ventas_por_cliente[cliente_id]['productos_diferentes'].add(venta['producto_id'])
        
        # Actualizar última compra
        if venta['fecha'] > ventas_por_cliente[cliente_id]['ultima_compra']:
            ventas_por_cliente[cliente_id]['ultima_compra'] = venta['fecha']
    
    # Convertir sets a conteos
    for cliente_id in ventas_por_cliente:
        ventas_por_cliente[cliente_id]['productos_diferentes'] = len(
            ventas_por_cliente[cliente_id]['productos_diferentes']
        )
    
    print("👥 ANÁLISIS DE CLIENTES")
    print("=" * 30)
    
    # Top clientes por ingresos
    clientes_ordenados = sorted(
        ventas_por_cliente.items(),
        key=lambda x: x[1]['ingresos'],
        reverse=True
    )
    
    print("🏆 TOP 5 CLIENTES POR INGRESOS:")
    print("-" * 35)
    
    for i, (cliente_id, datos) in enumerate(clientes_ordenados[:5], 1):
        info_cliente = clientes_dict.get(cliente_id, {})
        nombre = info_cliente.get('nombre', 'Desconocido')
        tipo = info_cliente.get('tipo', 'N/A')
        
        valor_promedio = datos['ingresos'] / datos['transacciones']
        
        print(f"{i}. {nombre} ({cliente_id})")
        print(f"   Tipo: {tipo}")
        print(f"   Ingresos totales: ${datos['ingresos']:,.2f}")
        print(f"   Transacciones: {datos['transacciones']}")
        print(f"   Valor promedio: ${valor_promedio:,.2f}")
        print(f"   Productos diferentes: {datos['productos_diferentes']}")
        print(f"   Última compra: {datos['ultima_compra']}")
        print()
    
    # Análisis por tipo de cliente
    ventas_por_tipo = {}
    
    for cliente_id, datos_venta in ventas_por_cliente.items():
        info_cliente = clientes_dict.get(cliente_id, {})
        tipo = info_cliente.get('tipo', 'Desconocido')
        
        if tipo not in ventas_por_tipo:
            ventas_por_tipo[tipo] = {
                'clientes': 0,
                'ingresos': 0,
                'transacciones': 0
            }
        
        ventas_por_tipo[tipo]['clientes'] += 1
        ventas_por_tipo[tipo]['ingresos'] += datos_venta['ingresos']
        ventas_por_tipo[tipo]['transacciones'] += datos_venta['transacciones']
    
    print("📊 ANÁLISIS POR TIPO DE CLIENTE:")
    print("-" * 32)
    
    for tipo, datos in ventas_por_tipo.items():
        ingresos_promedio = datos['ingresos'] / datos['clientes']
        transacciones_promedio = datos['transacciones'] / datos['clientes']
        
        print(f"Tipo: {tipo}")
        print(f"  Clientes: {datos['clientes']}")
        print(f"  Ingresos totales: ${datos['ingresos']:,.2f}")
        print(f"  Ingresos promedio por cliente: ${ingresos_promedio:,.2f}")
        print(f"  Transacciones promedio: {transacciones_promedio:.1f}")
        print()
    
    return ventas_por_cliente, ventas_por_tipo

# Ejecutar análisis de clientes
ventas_por_cliente, ventas_por_tipo = analizar_clientes(ventas, clientes)

## 5. 📈 Generación de Insights y Recomendaciones

In [ ]:
# TODO: Generar insights y recomendaciones

def generar_insights(ventas, ventas_por_producto, ventas_por_region, ventas_por_tipo):
    """Genera insights y recomendaciones basados en el análisis"""
    
    print("🧠 INSIGHTS Y RECOMENDACIONES")
    print("=" * 35)
    
    # Insight 1: Producto más rentable
    producto_top = max(ventas_por_producto.items(), key=lambda x: x[1]['ingresos'])
    producto_id, datos_producto = producto_top
    nombre_producto = productos.get(producto_id, {}).get('nombre', 'Desconocido')
    
    print("💡 INSIGHT #1: Producto Estrella")
    print(f"   {nombre_producto} ({producto_id}) genera el {(datos_producto['ingresos']/sum(venta['total'] for venta in ventas)*100):.1f}% de los ingresos")
    print(f"   Recomendación: Aumentar stock y marketing para este producto")
    print()
    
    # Insight 2: Región con mejor rendimiento
    region_top = max(ventas_por_region.items(), key=lambda x: x[1]['ingresos'])
    region_nombre, datos_region = region_top
    
    print("💡 INSIGHT #2: Región Líder")
    print(f"   La región {region_nombre} representa el {(datos_region['ingresos']/sum(venta['total'] for venta in ventas)*100):.1f}% de los ingresos")
    print(f"   Recomendación: Replicar estrategias exitosas de {region_nombre} en otras regiones")
    print()
    
    # Insight 3: Tipo de cliente más valioso
    tipo_top = max(ventas_por_tipo.items(), key=lambda x: x[1]['ingresos'])
    tipo_nombre, datos_tipo = tipo_top
    
    print("💡 INSIGHT #3: Segmento Prioritario")
    print(f"   Clientes {tipo_nombre} generan ${datos_tipo['ingresos']:,.2f} en ingresos")
    print(f"   Promedio por cliente: ${datos_tipo['ingresos']/datos_tipo['clientes']:,.2f}")
    print(f"   Recomendación: Desarrollar programa de fidelización para clientes {tipo_nombre}")
    print()
    
    # Insight 4: Oportunidades de crecimiento
    productos_bajo_rendimiento = []
    ingresos_total = sum(venta['total'] for venta in ventas)
    
    for producto_id, datos in ventas_por_producto.items():
        participacion = (datos['ingresos'] / ingresos_total) * 100
        if participacion < 15:  # Menos del 15% de participación
            productos_bajo_rendimiento.append((producto_id, participacion))
    
    if productos_bajo_rendimiento:
        print("💡 INSIGHT #4: Oportunidades de Crecimiento")
        for producto_id, participacion in productos_bajo_rendimiento:
            nombre = productos.get(producto_id, {}).get('nombre', 'Desconocido')
            print(f"   {nombre}: {participacion:.1f}% de participación")
        print(f"   Recomendación: Evaluar estrategias de marketing para productos con baja participación")
        print()
    
    # Resumen ejecutivo
    print("📋 RESUMEN EJECUTIVO")
    print("=" * 25)
    total_ingresos = sum(venta['total'] for venta in ventas)
    total_transacciones = len(ventas)
    ticket_promedio = total_ingresos / total_transacciones
    
    print(f"• Ingresos del período: ${total_ingresos:,.2f}")
    print(f"• Número de transacciones: {total_transacciones}")
    print(f"• Ticket promedio: ${ticket_promedio:,.2f}")
    print(f"• Productos activos: {len(ventas_por_producto)}")
    print(f"• Regiones atendidas: {len(ventas_por_region)}")
    print(f"• Clientes activos: {len(set(venta['cliente_id'] for venta in ventas))}")

# Generar el reporte final
generar_insights(ventas, ventas_por_producto, ventas_por_region, ventas_por_tipo)

## 6. 💾 Exportar Resultados

In [ ]:
# TODO: Exportar resultados a archivos

def exportar_reporte_resumen():
    """Exporta un reporte resumen a archivo de texto"""
    
    try:
        # Crear directorio de resultados si no existe
        if not os.path.exists('resultados'):
            os.makedirs('resultados')
        
        with open('resultados/reporte_resumen.txt', 'w', encoding='utf-8') as file:
            file.write(f"REPORTE DE ANÁLISIS DE DATOS\n")
            file.write(f"{EMPRESA}\n")
            file.write(f"Período: {PERIODO_ANALISIS}\n")
            file.write(f"Generado: {FECHA_REPORTE}\n")
            file.write("=" * 50 + "\n\n")
            
            # Métricas generales
            total_ingresos = sum(venta['total'] for venta in ventas)
            total_transacciones = len(ventas)
            ticket_promedio = total_ingresos / total_transacciones
            
            file.write("MÉTRICAS GENERALES\n")
            file.write("-" * 20 + "\n")
            file.write(f"Ingresos totales: ${total_ingresos:,.2f}\n")
            file.write(f"Transacciones: {total_transacciones}\n")
            file.write(f"Ticket promedio: ${ticket_promedio:,.2f}\n")
            file.write(f"Productos vendidos: {len(ventas_por_producto)}\n")
            file.write(f"Clientes activos: {len(set(venta['cliente_id'] for venta in ventas))}\n\n")
            
            # Top productos
            file.write("TOP PRODUCTOS\n")
            file.write("-" * 15 + "\n")
            productos_ordenados = sorted(
                ventas_por_producto.items(),
                key=lambda x: x[1]['ingresos'],
                reverse=True
            )
            
            for i, (producto_id, datos) in enumerate(productos_ordenados[:3], 1):
                nombre = productos.get(producto_id, {}).get('nombre', 'Desconocido')
                file.write(f"{i}. {nombre}: ${datos['ingresos']:,.2f}\n")
            
        print("✅ Reporte resumen exportado a 'resultados/reporte_resumen.txt'")
        
    except Exception as e:
        print(f"❌ Error exportando reporte: {e}")

def exportar_datos_procesados():
    """Exporta datos procesados en formato JSON"""
    
    try:
        datos_procesados = {
            'resumen': {
                'fecha_reporte': FECHA_REPORTE,
                'periodo': PERIODO_ANALISIS,
                'total_ingresos': sum(venta['total'] for venta in ventas),
                'total_transacciones': len(ventas),
                'productos_activos': len(ventas_por_producto),
                'clientes_activos': len(set(venta['cliente_id'] for venta in ventas))
            },
            'ventas_por_producto': ventas_por_producto,
            'ventas_por_region': {
                region: {
                    'ingresos': datos['ingresos'],
                    'transacciones': datos['transacciones'],
                    'clientes_unicos': len(datos['clientes_unicos'])
                } for region, datos in ventas_por_region.items()
            },
            'ventas_por_tipo_cliente': ventas_por_tipo
        }
        
        with open('resultados/datos_procesados.json', 'w', encoding='utf-8') as file:
            json.dump(datos_procesados, file, indent=2, ensure_ascii=False)
        
        print("✅ Datos procesados exportados a 'resultados/datos_procesados.json'")
        
    except Exception as e:
        print(f"❌ Error exportando datos: {e}")

# Exportar resultados
exportar_reporte_resumen()
exportar_datos_procesados()

print("\n🎉 ¡Análisis completado exitosamente!")
print("📁 Los resultados han sido guardados en la carpeta 'resultados/'")

## 🎯 Reflexión Final

### ¿Qué hemos logrado?

En este proyecto capstone has integrado exitosamente:

✅ **Manejo de archivos**: CSV, JSON y TXT  
✅ **Procesamiento de datos**: Limpieza y transformación  
✅ **Análisis estadístico**: Métricas y agregaciones  
✅ **Estructuras de datos**: Listas, diccionarios, sets  
✅ **Control de flujo**: Condicionales y bucles  
✅ **Funciones**: Código modular y reutilizable  
✅ **Manejo de errores**: Código robusto  
✅ **Exportación**: Resultados en múltiples formatos  

### Habilidades Desarrolladas

- **Pensamiento analítico** para extraer insights
- **Programación estructurada** con buenas prácticas
- **Resolución de problemas** complejos paso a paso
- **Comunicación de resultados** de manera clara

### Próximos Pasos

Con estas bases sólidas, estás preparado para:
- Trabajar con librerías como pandas y numpy
- Crear visualizaciones con matplotlib/seaborn
- Desarrollar modelos de machine learning
- Construir dashboards interactivos

---

**¡Felicidades por completar el curso! Ahora eres un Python Data Analyst 🚀**